In [2]:
# SKT Brain github 주소는 다음과 같습니다. https://github.com/SKTBrain/KoBERT

# !pip install mxnet
# !pip install gluonnlp pandas tqdm
# !pip install sentencepiece
# !pip install transformers==3 # 최신 버전으로 설치하면 "Input: must be Tensor, not str" 라는 에러 발생
# !pip install torch

# !pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

# kobert 
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

# transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup


     |████████████████████████████████| 46.9 MB 120 kB/s  eta 0:00:01
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.16
    Uninstalling graphviz-0.16:
      Successfully uninstalled graphviz-0.16
     |████████████████████████████████| 344 kB 29.3 MB/s eta 0:00:01
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp36-cp36m-linux_x86_64.whl size=468329 sha256=d6f817436ff3059fff30fe904cb8d55ebe772b00f4a7826aadfa7b2f15904d95
  Stored in directory: /home/adminuser/.cache/pip/wheels/62/62/9a/53be069ac8c9dde533dacce0e716193a4a43e87b5d37f5008c
Successfully built gluonnlp
     |████████████████████████████████| 754 kB 12.2 MB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 82.9 MB/s eta 0:00:01
ERROR: azureml-automl-dnn-nlp 1.29.0 has requirement transformers<=4.5.1,>=4.1.0, but you'll have transformers 3.0.0 which is incompatible.
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.10.2
    Uninstalling tokenizers-

In [3]:
# GPU 사용 시
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Bert모델, Voca 불러오기
bertmodel, vocab = get_pytorch_kobert_model()


[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [4]:
# 학습용 데이터셋 불러오기
import pandas as pd
# !wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1

# !wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1

In [4]:
#field_indices=[데이터 있는 열, 레이블링 된 열]
#num_discard_samples=헤더가 있으면 상위 row 1개 제거
# df_train = nlp.data.TSVDataset("ratings_train.txt?dl=1", field_indices=[1,2],num_discard_samples=1)
# df_test = nlp.data.TSVDataset("ratings_test.txt?dl=1", field_indices=[1,2],num_discard_samples=1)

In [5]:
# df_train

In [6]:
# # 질병명 라벨링
# from sklearn.preprocessing import LabelEncoder
# encoder = LabelEncoder()
# encoder.fit(new_data['질병명'])
# new_data['질병명'] = encoder.transform(new_data['질병명'])
# new_data.head()

# # 라벨링된 질병명 매핑 ex) {0: '비염', 1: '소화불량', 2: '수족냉증', 3: '식중독', 4: '질염'}
# mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))
# mapping

In [7]:
PATH = '../korean/'

In [8]:
new_data = pd.read_csv(PATH + "sentiment_sample.csv")
new_data.sample(n=10)

,Title_Article,sentiment
18424,일 만에 만잔 팔린 스벅 봄맞이 음료가 돌아온다스타벅스커피코리아 일 슈크림 라떼 ...,1
13208,산은 조원 규모 재무안정 동행프로그램 종 출시서울 신효령 산업은행은 신종 코로나바이...,1
3365,코스피 상승 마감 선 회복종합코스피 소폭 하락 출발 서울 박동주 코스피가 소폭 하락...,0
19740,마음여행 지구 두바퀴반 세계일주 가장 필요한건 용기와 결단도전하지 않으면 아무 일도...,0
11043,우리들제약 팜젠사이언스 로 사명 변경제 기 주총서 사명 변경 및 박성문 신규 감사 ...,0
18698,중국판 백신여권 한국에서도 통할까 방역당국 답변은머니 안정준 서울 뉴스 안은나 손영...,0
233,KB운용 SK증권에서 SKIET 청약하고 KBSTAR ETF 주 받자KB자산운용이 ...,0
8197,코스피 시대 쩐의 전쟁 화수분 동학개미 VS 내다파는 기관개인투자자 하루 조 역대 ...,1
5837,아파트 단지에 영화관이 현대건설메가박스 업무협약 체결코로나 시대 맞아 변화된 트렌드...,1
4437,삼성엔지니어링 유가 상승으로 수주 개선 목표가 신한금융투자 신한금융투자는 일 삼성엔...,1


In [9]:
data_list = []
for q, label in zip(new_data['Title_Article'], new_data['sentiment'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [10]:
print(data_list[0])
print(data_list[60])

['경기 회복 기대감에 상가 거래량 증가 힐스 에비뉴 신방화역 주목최근 백신 접종률이 증가하면서 경기가 회복 될 것이라는 기대감 속에 부동산 시장도 활기를 찾는 모습이다 실제로 최근 전국 부동산 거래도 늘고 있다 실거래 자료를 보면 올해 월 전국 상업 업무용 부동산 거래량은 총 건으로 전년동월 년 월 건 대비 약 증가했다 이러한 가운데 현대건설이 서울 역세권 입지에 신규 상가를 공급해 주목 받고 있다 힐스 에비뉴 신방화역 은 서울시 강서구 방화동 일원에서 프리미엄 브랜드 힐스 에비뉴 로 공급되며 지하 층 지상 층 총 실 규모로 구성된다 지하철 호선 신방화역 바로 앞에 위치한 초역세권 대로변 상가로 조성돼 서울 강남과 여의도 김포공항 등에서 근무하는 직장인 유동인구가 풍부하다 실제로 서울시 지하철 승하차 인원 자료를 보면 지난해 월 기준 신방화역을 이용한 승하차 인원은 총 만 명으로 인근에 위치한 호선 송정역 이용객 만 명 호선 공항시장역 이용객 명을 웃돌았다 아울러 아파트 입주민을 비롯해 주변으로 풍부한 주거 수요를 갖춘다 마곡엠밸리 단지 세대 를 비롯해 마곡 힐스테이트 세대 마곡 푸르지오 세대 등 상업시설 반경 km 내에 약 만 세대가 넘는 대규모 주거타운이 조성돼 있다 여기에 방화뉴타운 초입에 위치해 총 만 여 세대의 주거 수요를 품을 전망이다 특히 대로변에 위치해 있어 도로 접근성이 우수하고 주변 단지 입주민의 유입이 수월하다 한편 견본주택은 서울시 강서구 마곡동 번지 층에 위치해 있다 ', '1']
['갑질논란 휘말리면 어쩌나 면접관 자기검열머니 이재윤 구인구직 매칭플랫폼 사람인은 개 기업을 대상으로 면접 갑질 논란 에 대한 우려를 하고 있냐는 설문조사를 실시한 결과 가 그렇다고 답했다고 일 밝혔다 면접관들은 면접 시 자기검열을 강화하고 있다 고 답했다 이유로는 회사 관리를 위해 복수응답 라는 답변이 단연 많았다 이어 좋은 인재를 뽑기 위해서 면접 갑질이 사회적인 이슈가 되어서 문제 발생 시 불이익을 받을 수 있어서 회사의 내부 방침이어서 지원자로부터 

In [46]:
type(data_list[0][1])

str

In [11]:

# Train / Test set 분리
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, random_state=42)
print("train shape is:", len(dataset_train))
print("test shape is:", len(dataset_test))


train shape is: 16000
test shape is: 4000


In [12]:
# kobert 입력 데이터로 만들기

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair) 

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [13]:
# Setting parameters
max_len = 64 # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 64
warmup_ratio = 0.1
num_epochs = 20 # 훈련 반복횟수
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

In [14]:
# 토큰화
# 기본 Bert tokenizer 사용
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [15]:
# data 패딩화
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [16]:
data_train[0]
#첫 번째는 패딩된 시퀀스
#두 번째는 길이와 타입에 대한 내용
#세 번재는 어텐션 마스크 시퀀스

(array([   2, 4243, 7147, 4197, 6398, 6579, 1651, 1931, 1931, 2034,  517,
        5713, 2287, 1316, 6527, 7970, 3634, 3042, 6999, 7147, 2804, 2426,
        3463, 7408, 4919,  638,  273, 4297, 6951, 7828, 1196,  993, 5424,
        6263, 4203, 6402, 6585, 1434, 5760, 4243, 7148,  517, 5712, 1946,
        2287, 6079, 1557, 4243, 6416, 4197, 6398, 6579, 1426, 7147, 4257,
        4243, 7147, 2536, 1864, 1255, 7869, 3803, 1162,    3], dtype=int32),
 array(64, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 0)

In [17]:
# torch 형식의 dataset 만들어주기 (pytorch용 DataLoader 사용)
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [18]:
# Kobert 학습모델 만들기

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 2, # softmax 사용 <- binary일 경우는 2
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)
      

In [55]:
#conda install pytorch torchvision cudatoolkit=10.1 -c pytorch

In [19]:
import torch
print(torch.cuda.device_count())

1


In [20]:
# Bert 모델 불러오기

model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

# optimizer설정 하고 schedule 설정(linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

# 옵티마이저 선언
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # softmax용 Loss Function 정하기 <- binary classification도 해당 loss function 사용 가능

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)


In [21]:
# 학습 평가 지표인 accuracy 계산 -> 얼마나 타겟값을 많이 맞추었는가
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
  
train_dataloader

In [22]:
# 모델 학습 시작
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) # gradient clipping
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    

/anaconda/envs/azureml_py36_pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/250 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.6924657225608826 train acc 0.515625
epoch 1 batch id 201 loss 0.4936772286891937 train acc 0.6388370646766169
epoch 1 train acc 0.663875


/anaconda/envs/azureml_py36_pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/63 [00:00<?, ?it/s]

...

epoch 19 test acc 0.7986111111111112


  0%|          | 0/250 [00:00<?, ?it/s]

epoch 20 batch id 1 loss 0.0006921594613231719 train acc 1.0
epoch 20 batch id 201 loss 0.00035671930527314544 train acc 0.9996113184079602
epoch 20 train acc 0.999625


  0%|          | 0/63 [00:00<?, ?it/s]

epoch 20 test acc 0.798859126984127


In [30]:
# 테스트 문장 예측
test_sentence = "본격적으로 막이 오른 쌍용차 인수전이 SM그룹의 '깜짝 등판'으로 후끈 달아오른 가운데 당사자인 쌍용차 역시 예상외의 흥행에 미소 지으며 후속 작업을 준비하고 있다. 최근 쌍용차가 친환경차 전용공장 건설 계획을 밝히는 등 전기차 전환에 뒤늦게 속도를 내는 가운데 상당수의 투자자가 전기차 사업 확대를 목표로 인수 의향을 밝히고 나선 것도 긍정적이라는 분위기다."
test_label = 1 # 실제 값

unseen_test = pd.DataFrame([[test_sentence, test_label]], columns = [['title', 'cls']])
unseen_values = unseen_test.values
test_set = BERTDataset(unseen_values, 0, 1, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=5)

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    out = model(token_ids, valid_length, segment_ids)
    print(out)
    
model.eval() # 평가 모드로 변경
    
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)
    test_acc += calc_accuracy(out, label)
print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/anaconda/envs/azureml_py36_pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


  0%|          | 0/1 [00:00<?, ?it/s]

/anaconda/envs/azureml_py36_pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/63 [00:00<?, ?it/s]

epoch 20 test acc 3.195436507936508


In [33]:
# 감성값 예측하는 함수 만들기

import pandas as pd

# 위에서 설정한 tok, max_len, batch_size, device를 그대로 입력
# comment : 예측하고자 하는 텍스트 데이터 리스트
def getSentimentValue(comment, tok, max_len, batch_size, device):
    commnetslist = [] # 텍스트 데이터를 담을 리스트
    emo_list = [] # 감성 값을 담을 리스트
    for c in comment: # 모든 댓글
        commnetslist.append( [c, 5] ) # [댓글, 임의의 양의 정수값] 설정

    pdData = pd.DataFrame( commnetslist, columns = [['뉴스', '감성']] )
    pdData = pdData.values
    test_set = BERTDataset(pdData, 0, 1, tok, max_len, True, False) 
    test_input = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=5)

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_input):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length 
        # 이때, out이 예측 결과 리스트
        out = model(token_ids, valid_length, segment_ids)
        print(out)
        # e는 2가지 실수 값으로 구성된 리스트
        # 0번 인덱스가 더 크면 부정, 긍정은 반대
        for e in out:
            if e[0]>e[1]: # 부정
                value = 0
                emo_list.append("부정")
            else: #긍정
                value = 1
                emo_list.append("긍정")

    return emo_list # 텍스트 데이터에 1대1 매칭되는 감성값 리스트 반환

# input : 텍스트 데이터 리스트 외 KoBERT 설정 파라미터들
# output : 입력한 텍스트 데이터 리스트와 1대 1 매칭 되는 감성 값 리스트

In [49]:

def news():
    
    comment = []
    comment.append(input("원하는 기사를 입력하세요"))

    for c in comment:
        print(f'\n기사 : {c}\n')
        
    getSentimentValue(comment, tok, max_len, batch_size, device)

In [54]:
news()

원하는 기사를 입력하세요삼성전자와 SK하이닉스 등 대형 반도체주 강세에 코스피가 3,230선을 회복했습니다.  코스피지수는 전 거래일보다 0.44% 오른 3,237.14에 거래를 마쳤습니다.  삼성전자는 외국인이 6,000억 원 넘게 사들이면서 13거래일 만에 8만 원 선을 회복했고, SK하이닉스도 3% 넘게 올랐습니다.  다만 중국에서 규제 리스크가 불거진 게임주가 약세를 보이며 상승폭을 제한했습니다.

기사 : 삼성전자와 SK하이닉스 등 대형 반도체주 강세에 코스피가 3,230선을 회복했습니다.  코스피지수는 전 거래일보다 0.44% 오른 3,237.14에 거래를 마쳤습니다.  삼성전자는 외국인이 6,000억 원 넘게 사들이면서 13거래일 만에 8만 원 선을 회복했고, SK하이닉스도 3% 넘게 올랐습니다.  다만 중국에서 규제 리스크가 불거진 게임주가 약세를 보이며 상승폭을 제한했습니다.

tensor([[-3.7498,  4.4319]], device='cuda:0', grad_fn=<AddmmBackward>)
